### imports

In [1]:
import folium

import openrouteservice as ors

### setup: api-key, client, Karte, Daten

In [2]:
api_key = '5b3ce3597851110001cf6248d2f185cd80644b67a7ca91371d7b6419'
host = "http://localhost:8082/ors"   # falls wir den openrouteservice lokal laufen lassen


client = ors.Client(key=api_key)
m = folium.Map(location=([49.411085, 8.685894]), zoom_start=12)

# Wohnungen
wohnungen = {
    'Hintergasse 4, Dossenheim': {'location': [8.677064, 49.452716,]},
    'Heckerstraße 59, Kirchheim': {'location': [8.66317, 49.378077]},
    'Kreuzstraße 3, Wieblingen' : {'location': [8.648257, 49.426977]}
}

### Was ist in 15 Minuten zu Fuß erreichbar?

In [3]:
profil = 'foot-walking'
reichweite = [900] # 900/60 = 15

for name, info in wohnungen.items():
    info['iso'] = client.isochrones(
                    locations = [info['location']],
                    profile=profil, 
                    range=reichweite
                  ) 
    
    folium.features.GeoJson(info['iso']).add_to(m)
    
    folium.map.Marker(
        list(reversed(info['location'])), # folium nutzt lat/lon :(
        icon=folium.Icon(color='lightgray', icon_color='#cc0000', icon='home', prefix='fa'),
        popup=name
    ).add_to(m)

m

### Was wollen wir erreichen?

In [4]:
# POI Kategorien: https://github.com/GIScience/openrouteservice-docs#places-response
pois = {
    'Haltestelle': [597, 604, 605],
    'Supermarkt': [518],
    'Fahrradladen': [429]
}

for name, info in wohnungen.items():
    info['poi_typen'] = dict()
    print(f"\nWohnung {name}:")
    
    for poi_typ, ids in pois.items():
        info['poi_typen'][poi_typ] = dict()
        info['poi_typen'][poi_typ]['geojson']= client.places(
                                                    request = 'pois',
                                                    sortby = 'distance',
                                                    geojson = info['iso']['features'][0]['geometry'],
                                                    filter_category_ids = ids
                                                )[0]['features']
        
        print(f"\t{poi_typ}: {len(info['poi_typen'][poi_typ]['geojson'])}")


Wohnung Hintergasse 4, Dossenheim:
	Haltestelle: 3
	Supermarkt: 3
	Fahrradladen: 1

Wohnung Heckerstraße 59, Kirchheim:
	Haltestelle: 5
	Supermarkt: 3
	Fahrradladen: 1

Wohnung Kreuzstraße 3, Wieblingen:
	Haltestelle: 2
	Supermarkt: 4
	Fahrradladen: 3


### Wie weit ist das weg?

In [5]:

# Icons für Visualisierung
style_dict = {'Haltestelle': 'subway',
              'Supermarkt': 'shopping-cart',
              'Fahrradladen': 'bicycle'
             }

for info in wohnungen.values():
    for kategorie, pois in info['poi_typen'].items():
        pois['entfernungen'] = []
        for poi in pois['geojson']:
            poi_coords = poi['geometry']['coordinates']
            route = client.directions(
                                profile = profil,
                                instructions = "false",
                                format = "geojson",
                                coordinates =[info['location'], poi_coords]
                                )
            
            folium.features.GeoJson(route).add_to(m)
            folium.map.Marker(list(reversed(poi_coords)),
                              icon=folium.Icon(color='white', icon_color='#1a1aff', icon=style_dict[kategorie], prefix='fa'),
                              popup=poi['properties']['osm_tags']['name'],
                             ).add_to(m)
            
            entfernung = route['features'][0]['properties']['summary']['duration']
            pois['entfernungen'].append(entfernung) # Record durations of routes
        
m

### Wie weit ist das im Schnitt?

In [6]:
for name, info in wohnungen.items():
    info['shortest_sum'] = sum([min(kategorie['entfernungen']) for kategorie in info['poi_typen'].values()])
    print(f"Wohnung {name} : {(info['shortest_sum']/60)/3:.2f} min im Schnitt")

Wohnung Hintergasse 4, Dossenheim : 6.64 min im Schnitt
Wohnung Heckerstraße 59, Kirchheim : 4.47 min im Schnitt
Wohnung Kreuzstraße 3, Wieblingen : 4.63 min im Schnitt
